# Poisson editing
Adapted by Nicolas Papadakis (IMB) and Charles Dossal (INSA Toulouse) from works by William Emmanuel and Pierre Bénard (LaBRI)

## Introduction
The goal of this assignment is to apply the Poisson editing algorithm [1] for image blending.

In the following, $T$ is a target image,  $S$ a source image,  and a binary mask representing an area $\Omega$ of $S$ to copy in $T$. All images are defined on the same domain $D$ of sizee $M\times N$.

The idea of [1] is to copy the spatial gradients $\nabla S$ of the source image inside $T$, and not the color values $S$. As illustrated below, this gives more realistic blendings.




<table align="center"><tr><td><img src="./img/target_Boat.png" style="width: 200px;"></td><td><img src="./img/source_Kraken.png" style="width: 200px;"></td><td><img src="./img/mask_Kraken.png" style="width: 200px;"></td><td><img src="./img/naive.png" style="width: 200px;"></td><td><img src="./img/poisson_blending.png" style="width: 200px;"></td></tr>
<tr><td>Target $T$</td><td>Source $S$</td><td>Mask</td><td>Naive copy/paste of $S$</td><td>Poisson blending [1]: copy of $\nabla S$</td></tr>
</table>

To realize such blending, we find an image $u$ solution of:

$$\min_u \int_\Omega ||\nabla u-\nabla S||^2,$$
under the constraint $u_{D\backslash \Omega}=T$.

This problem can be stated as follow
\begin{equation*}
\min_u \int_\Omega ||\nabla u-\nabla S||^2+\iota_{K}(u)
\end{equation*}
where $K$ is the set of images which coincide with the target out of the mask. 
We can observe that the set $K$ is a closed convex set... Why ?

In this setting we can use the projected gradient which is a particular case of the Forward backward algorithm to solve this problem of fusion of images.

We will study two examples but you can try others on your own.
The data can be found at the following adress :

http://dl.free.fr/rm.pl?h=bOSoNRP4l&i=93426714&s=mwWbk2t533WxFMxD47ERDuR9MlkDPwc8


In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
import matplotlib.pyplot as plt
import requests
from panel.pane import LaTeX
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from io import BytesIO


In [ ]:
caselist=['Kraken', 'MonaLisa']

Data are available online but you have downloaded them, you can work online changing the value of the variable "local" in the next cell.

In [ ]:
local=1
def chargeData(name):
    if local:
        if name=='Kraken':
            target=np.array(Image.open("./img/target_Boat.png")).astype(float)
            source=np.array(Image.open("./img/source_Kraken.png")).astype(float)
            mask2=np.array(Image.open("./img/mask_Kraken.png")).astype(float)/255
        if name=='MonaLisa':
            target=np.array(Image.open("./img/Joconde.jpg")).astype(float)
            source=np.array(Image.open("./img/source_Heisenberg.jpeg")).astype(float)
            mask2=np.array(Image.open("./img/mask_joconde.jpeg")).astype(float)/255
    else:
        if name=='Kraken':
            url='https://plmlab.math.cnrs.fr/dossal/charlesdossalnesterov/raw/master/img/target_Boat.png?inline=false'
            response = requests.get(url)
            target=np.array(Image.open(BytesIO(response.content))).astype(float)
            url='https://plmlab.math.cnrs.fr/dossal/charlesdossalnesterov/raw/master/img/source_Kraken.png?inline=false'
            response = requests.get(url)
            source=np.array(Image.open(BytesIO(response.content))).astype(float)
            url='https://plmlab.math.cnrs.fr/dossal/charlesdossalnesterov/raw/master/img/mask_Kraken.png?inline=false'
            response = requests.get(url)
            mask2=np.array(Image.open(BytesIO(response.content))).astype(float)/255
        if name=='MonaLisa':
            url='https://plmlab.math.cnrs.fr/dossal/charlesdossalnesterov/raw/master/img/Joconde.jpg?inline=false'
            response = requests.get(url)
            target=np.array(Image.open(BytesIO(response.content))).astype(float)
            url='https://plmlab.math.cnrs.fr/dossal/charlesdossalnesterov/raw/master/img/source_Heisenberg.jpeg?inline=false'
            response = requests.get(url)
            source=np.array(Image.open(BytesIO(response.content))).astype(float)
            url='https://plmlab.math.cnrs.fr/dossal/charlesdossalnesterov/raw/master/img/mask_joconde.jpeg?inline=false'
            response = requests.get(url)
            mask2=np.array(Image.open(BytesIO(response.content))).astype(float)/255
    return target,source,mask2

Have a look to the two set of data.

In [ ]:
target,source,mask2=chargeData('Kraken')
optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask2*255).astype('uint8')).opts(**optionsGray))

In the following we are giving some discrete gradient and associated divergence.

In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return y
def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

Define the two functions Projection and Gradient that will be necessary to compute the projected gradient. What is the Lipschitz constant of the gradient of $x\mapsto \Vert\nabla x -y\Vert_2^2$ ?

In [ ]:
def Proj(im,ma,iref):

    return res
def GradientFonc(x,y):
    
    return res

In the first step we divide the source and the target into the three chanels

In [ ]:
target0=target[:,:,0]
source0=source[:,:,0]
target1=target[:,:,1]
source1=source[:,:,1]
target2=target[:,:,2]
source2=source[:,:,2]

Compute then a naive fusion with a simple projection.

Write a function  FBPoissonEditing that compute the projected gradient on a grayscale image (single color chanel). Don't forget to clip the image at the end.
The function must return the last iterate of the sequence and a curve of the values of iterates (that can be sampled for example with only at most 100 or 200 values) 

In [ ]:
def FBPoissonEditing(targ,sour,ma,step,Niter):
    
    return np.clip(x,0,255),f[10:]

Test the function with a step smaller than $1/4$ and 1000 iterations and compare the result with a naive approach. To get the result on a color image, the previous algorithm must be used on each color channel and the 3 output must be gather in a single color image.

Using panel, create a dashboard to perform the fusion in real time. The output may be four figures with the source, the target, the fusion and the curve of the decay of the function to minimize. 
The step in the gradient descend used in the algorithm will be $self.step/8$. Hence when the variable step will be set to 1, the step in the gradient descent will be exactly equal to the Lipschitz constant of the gradient of $f$.

In [ ]:
class FBFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(1,bounds=(0.1,4))
    def view(self):
        
        return 

Try the function. 

In [ ]:
fbfusion= FBFusion()
pn.Row(fbfusion.param,fbfusion.view)

Perform a Fusion using FISTA with a parameter $\alpha$ and create the associated dashboard. 


In [ ]:
def FISTAPoissonEditing(targ,sour,ma,step,alpha,Niter):
    
    return np.clip(x,0,255),f[10:]

In [ ]:
class FISTAFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(1,bounds=(0.1,4))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        
        return 

In [ ]:
fistafusion=FISTAFusion()
pn.Row(fistafusion.param,fistafusion.view)

Compare in a third dashboard the difference between FB and FISTA.
Are the limit on the step the same ?

In [ ]:
class FBvsFISTAFusion(param.Parameterized):
    case = param.ObjectSelector(default='Kraken',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(1,bounds=(0.1,4))
    alpha = param.Number(3,bounds=(2,15))
    def view(self):
        
        return 

In [ ]:
fbvsfista=FBvsFISTAFusion()
pn.Row(fbvsfista.param,fbvsfista.view)